In [1]:
import ray
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.ppo import APPOTrainer
import argparse
from ray.tune.registry import register_env
from pycigar.utils.registry import make_create_env
import yaml
import time
from pycigar.utils.input_parser import input_parser

SAVE_RATE = 2

"""
Parser to pass argument from terminal command
--run: RL algorithm, ex. PG, PPO, IMPALA
--stop: stop criteria of experiment. The experiment will stop when mean reward reach to this value.
Example of terminal command:
  > python single_relative_discrete_2_lr.py --run PPO --stop 0
"""
parser = argparse.ArgumentParser()
parser.add_argument("--run", type=str, default="PPO")  # try PG, PPO, IMPALA
parser.add_argument("--stop", type=int, default=0)

"""
Load the scenarios configuration file. This file contains the scenario information
for the experiment.
"""
sim_params = input_parser('ieee37busdata')

"""
Register the environment to OpenGymAI. This is necessary, RLlib can find the new environment
with string name env_name_v:version:, ex. SingleRelativeDiscreteCoopEnv_v0.
env_name: name of environment being used.
sim_params: simulation params, it is the scenario configuration.
simulator: the simulator being used, ex. opendss, gridlabd...
tracking_ids: list of ids of devices being tracked during the experiment.
"""

pycigar_params = {"exp_tag": "cooperative_multiagent_ppo",
                  "env_name": "CentralControlPVInverterEnv",
                  "sim_params": sim_params,
                  "simulator": "opendss",
                  "tracking_ids": ['adversary_inverter_s701a', 'creg1a', 'creg1c']}
"""
call function make_create_env() to register the new environment to OpenGymAI.
create_env() is a function to create new instance of the environment.
env_name: the registered name of the new environment.
"""
create_env, env_name, create_test_env, test_env_name = make_create_env(params=pycigar_params, version=0)
register_env(env_name, create_env)
register_env(test_env_name, create_test_env)

test_env = create_test_env()
obs_space = test_env.observation_space  # get the observation space, we need this to construct our agent(s) observation input
act_space = test_env.action_space  # get the action space, we need this to construct our agent(s) action output


In [2]:
obs = test_env.reset()
reward = 0
done = False
while not done:
    # for each observation, let the policy decides what to do
    obs, r, done, _ = test_env.step(3)
    reward += r
# plot the result. This will be saved in ./results
#test_env.plot(pycigar_params['exp_tag'], env_name, 0, reward)

In [3]:
test_env.plot(pycigar_params['exp_tag'], env_name, 0, reward)

In [ ]:
obs = test_env.reset()
reward = 0
done = False
while not done:
    # for each observation, let the policy decides what to do
    obs, r, done, _ = test_env.step(3)
    reward += r
# plot the result. This will be saved in ./results
#test_env.plot(pycigar_params['exp_tag'], env_name, 0, reward)

In [ ]:
env = create_env()
obs_space = env.observation_space  # get the observation space, we need this to construct our agent(s) observation input
act_space = env.action_space  # get the action space, we need this to construct our agent(s) action output
obs = env.reset()
reward = 0
done = False
while not done:
    # for each observation, let the policy decides what to do
    obs, r, done, _ = env.step(3)
    reward += r
# plot the result. This will be saved in ./results
#test_env.plot(pycigar_params['exp_tag'], env_name, 0, reward)

In [3]:
test_env.get_pycigar_output_specs()

{'Time Steps': 20,
 'Time Step Size (s)': 1,
 'allMeterVoltages': {'Min': [107.41153627738623,
   121.59197149085595,
   121.54049116071273,
   121.6756774911459,
   121.79128874339254,
   121.73202062153743,
   121.68374772791883,
   121.67702987815662,
   121.75910737682379,
   121.7274466403199,
   121.71696552520032,
   121.8000920092316,
   121.70296271166518,
   121.7250442321119,
   121.77317761219149,
   121.61598140717675,
   121.00514520111494,
   121.66135724042283,
   121.03308650450052,
   121.05259778357215,
   122.51242922433715,
   120.67162120515118,
   122.61791941945708,
   121.10636334619548,
   121.28088321868807,
   122.74339058809497],
  'Mean': [116.23996674935866,
   122.79777197284722,
   122.89083450245683,
   122.94689861524715,
   123.05261106157297,
   122.93308165868565,
   122.88586687306376,
   122.94858716980485,
   122.87169678629056,
   122.928377352277,
   122.91485347915565,
   122.95627004063005,
   122.8671873971783,
   122.82176298644603,
   122

In [5]:
directory = '/Users/toanngo/Documents/GitHub/ceds-cigar/rl/data/ieee37busdata/ieee37.dss'
dss.run_command('Redirect '+ directory)

''